In [1]:
import sys
sys.path.append("../") 

import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

import LISA as l

import Wavelet as wv
import Glitch as gl
import Burst as bu
import Burst_Slow as buS
import MCMC_tools as mct

# constants
mHz = 1.0e-3
Hour = 3600.

TabError: inconsistent use of tabs and spaces in indentation (Burst.py, line 282)

In [ ]:
Week = 3600.*24.*7.
dt   = 15.
Tobs = 2**np.ceil(np.log2(Week/dt))*dt

# print(np.log2(Tobs/dt))
# print(Tobs/dt)

# set up the LISA orbit
orb = l.Orbit(Tobs, dt=dt)
t = np.arange(0.0, orb.Tobs, orb.dt) # set up the time of this orbit
h = np.zeros(len(t))

print("Nyquist Freq........ {} mHz".format(orb.f_ny*1000.))

# set up parameters of wavelet
A    = 2.0e-21
f0   = 3.0*mHz
tau  = 10.5*Hour
t0   = 0.5*orb.Tobs
phi0 = 1.0
Q = 2*np.pi*f0*tau

print("Q................... {}".format(Q))
print("tau................. {} s".format(tau))
print("dt.................. {} s".format(orb.dt))

wave = wv.Wavelet(A, f0, tau, t0, phi0, orb)
wave.calc_Psi()
wave.make_padded_Psi(t) # drop in the wavelet time series into the data stream
h = h + wave.Psi_padded

In [ ]:
# set up parameters of wavelet
epsil = 0. # polarization parameter

wave2 = wv.Wavelet(epsil*A, f0, tau, t0, phi0, orb)
wave2.calc_Psi()
wave2.make_padded_Psi(t)

# sky angles and polarization angles.
phi   = np.pi*1.
theta = np.pi*1.
psi   = np.pi*0.2

# gw = bu.GW_glitch(wave, wave2, theta, phi, psi, orb, Flag_fast)
# gw.construct_detector_tensor()
# gw.calculate_strain()
# if (Flag_fast == 0):
#     tdi_GW = gw.construct_TDI(t, orb)
# else:
#     tdi_GW = gw.construct_TDI_FAST(orb)

gw_slow = buS.GW_glitch(wave, wave2, theta, phi, psi, orb)
gw_slow.construct_detector_tensor()
gw_slow.calculate_strain()
tdi_GW_slow = gw_slow.construct_TDI(t, orb)

In [ ]:
gw = bu.GW_glitch(wave, wave2, theta, phi, psi, orb)
gw.construct_detector_tensor()
gw.calculate_strain()
tdi_GW = gw_slow.construct_TDI(t, orb)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.set_xlabel(r'f [mHz]', fontsize=16)
ax.set_ylabel(r'$|\widetilde{TDI}|$', fontsize=16)

ax.tick_params(axis='both', which='major', labelsize=16)

# find an appropriate frequency range
ax.set_xlim(2.95, 3.05)
ax.set_ylim(1.0e-25, 1.0e-17)
ax.semilogy(tdi_GW_slow.freqs*1000, np.abs(tdi_GW_slow.X), label='Slow')

plt.show()

# plt.semilogy(tdi_GW_test.freqs, np.abs(tdi_GW_test.X))


# plt.plot(tdi_GW.freqs[mask], tdi_GW.X[mask].real)

# plt.plot(tdi_GW_test.freqs, tdi_GW_test.X.real, '-.')
# plt.xlim(0.00298,0.00302)
# plt.show()

# plt.plot(tdi_GW.freqs[mask], tdi_GW.X[mask].imag)

# plt.plot(tdi_GW_test.freqs, tdi_GW_test.X.imag, '-.')

